In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

Using TensorFlow backend.
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

0

In [2]:
main_path = r'f:\my\Prog\kaggle\Baydin'

In [3]:
import sys
sys.path.append(main_path)
from BayDS import *

In [4]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [5]:
#start here
data_folder = main_path+'/Data'
y = pd.read_pickle(f'{data_folder}/y.pkl')
X = pd.read_pickle(f'{data_folder}/X_encoded_scaled.pkl').astype(np.float32)
test = pd.read_pickle(f'{data_folder}/test_encoded_scaled.pkl').astype(np.float32)
gc.collect()

11

In [22]:
# Setting model_folder
model_name = 'keras-4Layer'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [7]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [8]:
y.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [35]:
from keras.regularizers import l1,l2

def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    x = numerical_inputs
    x = Dense(400, activation = 'relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(.3)(x)
    x = Dense(400, activation = 'relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(.3)(x)
    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)     
    out = Dense(1, activation = 'sigmoid')(x)    
    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 1024,
    'epochs': 100,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [33]:
import json
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)


In [36]:
NNModel_maker().save(f'{model_folder}/keras.mdl')

In [25]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2295671409795812128, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6696213545
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 943391395817768365
 physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:07:00.0, compute capability: 6.1"]

In [26]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [14]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [15]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [37]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Sat Sep  7 09:56:03 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/100
472432/472432 [==============================] - 22s 47us/step - loss: 0.9473 - val_loss: 0.5850
Epoch 2/100
472432/472432 [==============================] - 23s 49us/step - loss: 0.3459 - val_loss: 0.2370
Epoch 3/100
472432/472432 [==============================] - 24s 52us/step - loss: 0.1950 - val_loss: 0.1525
Epoch 4/100
472432/472432 [==============================] - 22s 46us/step - loss: 0.1387 - val_loss: 0.1360
Epoch 5/100
472432/472432 [==============================] - 24s 50us/step - loss: 0.1155 - val_loss: 0.0943
Epoch 6/100
472432/472432 [==============================] - 23s 48us/step - loss: 0.1024 - val_loss: 0.0869
Epoch 7/100
472432/472432 [==============================] - 24s 50us/step - loss: 0.0961 - val_loss: 0.0926
Epoch 8/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0925 - val_loss: 0.0866
Epoch 9/100
472432/472432 [======

472432/472432 [==============================] - 20s 43us/step - loss: 0.0745 - val_loss: 0.0775
Epoch 75/100
472432/472432 [==============================] - 21s 43us/step - loss: 0.0746 - val_loss: 0.0800
Epoch 76/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0743 - val_loss: 0.0774
Epoch 77/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0749 - val_loss: 0.0776
Epoch 78/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0743 - val_loss: 0.0774
Epoch 79/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0743 - val_loss: 0.0765
Epoch 80/100
472432/472432 [==============================] - 23s 48us/step - loss: 0.0745 - val_loss: 0.0763
Epoch 81/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0745 - val_loss: 0.0780
Epoch 82/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0747 - val_loss: 0.0800
Epoch 83/100
472432/472

472432/472432 [==============================] - 21s 44us/step - loss: 0.0724 - val_loss: 0.0938
Epoch 48/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0721 - val_loss: 0.0962
Epoch 49/100
472432/472432 [==============================] - 22s 47us/step - loss: 0.0716 - val_loss: 0.0907
Epoch 50/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0717 - val_loss: 0.0915
Epoch 51/100
472432/472432 [==============================] - 20s 43us/step - loss: 0.0721 - val_loss: 0.0907
Epoch 52/100
472432/472432 [==============================] - 20s 43us/step - loss: 0.0718 - val_loss: 0.0916
Epoch 53/100
472432/472432 [==============================] - 20s 43us/step - loss: 0.0714 - val_loss: 0.0901
Epoch 54/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0720 - val_loss: 0.0924
Epoch 55/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0712 - val_loss: 0.0936
Epoch 56/100
472432/472

472432/472432 [==============================] - 21s 44us/step - loss: 0.0758 - val_loss: 0.0917
Epoch 21/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0759 - val_loss: 0.0929
Epoch 22/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0753 - val_loss: 0.0941
Epoch 23/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0751 - val_loss: 0.0991
Epoch 24/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0749 - val_loss: 0.0916
Epoch 25/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0748 - val_loss: 0.0904
Epoch 26/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0748 - val_loss: 0.0926
Epoch 27/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0747 - val_loss: 0.0927
Epoch 28/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0742 - val_loss: 0.0947
Epoch 29/100
472432/472

472432/472432 [==============================] - 21s 44us/step - loss: 0.0711 - val_loss: 0.0936
Epoch 95/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0708 - val_loss: 0.0941
Epoch 96/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0704 - val_loss: 0.0909
Epoch 97/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0703 - val_loss: 0.0925
Epoch 98/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0704 - val_loss: 0.0964
Epoch 99/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0708 - val_loss: 0.0937
Epoch 100/100
118108/118108 [==============================] - 1s 11us/step
Fold 2. auc: 0.9266.
Fold 4 started at Sat Sep  7 11:50:30 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.9400 - val_loss: 0.4810
Epoch 2/100
472432/472432 [=========================

472432/472432 [==============================] - 21s 44us/step - loss: 0.0713 - val_loss: 0.0920
Epoch 68/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0716 - val_loss: 0.0884
Epoch 69/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0714 - val_loss: 0.0887
Epoch 70/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0715 - val_loss: 0.0916
Epoch 71/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0718 - val_loss: 0.0908
Epoch 72/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0714 - val_loss: 0.0896
Epoch 73/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0717 - val_loss: 0.0894
Epoch 74/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0714 - val_loss: 0.0880
Epoch 75/100
472432/472432 [==============================] - 21s 44us/step - loss: 0.0716 - val_loss: 0.0894
Epoch 76/100
472432/472

472432/472432 [==============================] - 21s 45us/step - loss: 0.0730 - val_loss: 0.0955
Epoch 41/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0731 - val_loss: 0.1033
Epoch 42/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0730 - val_loss: 0.0974
Epoch 43/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0729 - val_loss: 0.0956
Epoch 44/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0729 - val_loss: 0.0969
Epoch 45/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0732 - val_loss: 0.0978
Epoch 46/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0732 - val_loss: 0.0963
Epoch 47/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0727 - val_loss: 0.0963
Epoch 48/100
472432/472432 [==============================] - 21s 45us/step - loss: 0.0721 - val_loss: 0.0966
Epoch 49/100
472432/472

In [38]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [39]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv(f'{model_folder}/ieee_nn.csv', index=False)

In [40]:
result_dict_keras['prediction'].shape

(506691, 1)

In [41]:
test.shape

(506691, 789)

In [42]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)